https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49 https://data.cityofnewyork.us/Transportation/Traffic-Volume-Counts-2014-2018-/ertz-hr4r https://data.cityofnewyork.us/Housing-Development/Heating-Oil-Consumption-And-Cost-2010-2019-/bhwu-wuzu https://data.cityofnewyork.us/Housing-Development/Cooking-Gas-Consumption-And-Cost-2010-2019-/avhb-5jhc https://data.cityofnewyork.us/Housing-Development/Electric-Consumption-And-Cost-2010-March-2019-/jr24-e7cr https://data.cityofnewyork.us/Housing-Development/Heating-Gas-Consumption-And-Cost-2010-2019-/it56-eyq4 

https://data.cityofnewyork.us/Environment/Natural-Gas-Consumption-by-ZIP-Code-2010/uedp-fegm

boilers, building permits https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a description of permit types https://www1.nyc.gov/site/buildings/business/permit-type-and-job-status-codes.page

https://data.cityofnewyork.us/Environment/Steam-Consumption-by-ZIP-Code-2010/2pmt-skyq

land use data, PLUTO

traffic data 
https://data.cityofnewyork.us/Transportation/Traffic-Volume-Counts-2014-2018-/ertz-hr4r https://data.cityofnewyork.us/Transportation/Traffic-Volume-Counts-2011-2012-/wng2-85mv https://data.cityofnewyork.us/Transportation/Vehicle-Classification-Counts-2012-2013-/ae5u-upr6 https://data.cityofnewyork.us/Transportation/Vehicle-Classification-Counts-2011-2012-/t9nw-j73k https://data.cityofnewyork.us/Transportation/Vehicle-Classification-Counts-2014-2018-/96ay-ea4r https://data.cityofnewyork.us/dataset/NYC-Truck-Routes/spax-mybh

https://data.cityofnewyork.us/Transportation/MTA-Data/mmu8-8w8b

buses trucks https://data.cityofnewyork.us/dataset/NYC-Truck-Routes/spax-mybh roads https://data.cityofnewyork.us/City-Government/NYC-Street-Centerline-CSCL-/exjm-f27b

population density https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2 income

https://data.cityofnewyork.us/Environment/Landcover-Raster-Data-2010-3ft-Resolution/9auy-76zt


address info
https://data.cityofnewyork.us/City-Government/Property-Address-Directory/bc8t-ecyu

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sn
import plotly.express as px
import json
import datetime
#import geopandas as gpd
import fiona

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [ ]:
# https://data.cityofnewyork.us/Housing-Development/Heating-Gas-Consumption-And-Cost-2010-2019-/it56-eyq4
heating = pd.read_csv('Heating_Gas_Consumption_And_Cost__2010_-__2019_.csv')
heating

In [ ]:
ngc = pd.read_csv('Natural_Gas_Consumption_by_ZIP_Code_-_2010.csv')
ngc.head(100)

In [ ]:
ngc.groupby(['Zip Code']).count()

In [ ]:
ngc['zip2'] = ngc['Zip Code'].apply(lambda x : x[:5])
ngc

In [ ]:
ngc.groupby('zip2').count()


In [ ]:
ngc

In [ ]:
permits = pd.read_csv('DOB_Permit_Issuance.csv')
#permits.head(100)
permits.dtypes


In [ ]:
permits_bl = permits[permits['Permit Subtype'] == 'BL']
permits_bl['Job Start Date'] = pd.to_datetime(permits_bl['Job Start Date'])
permits_bl

In [ ]:
#permits_bl.count()
#import datetime
#permits_bl['Job Start Date'][7].year
permits_bl['year'] = permits_bl['Job Start Date'].apply(lambda x : x.year)
permits_bl['year'].hist(bins=100)

In [ ]:
permits_bl['Permit Type'].hist()

In [ ]:
addr = pd.read_csv('pad20a\\bobaadr.txt')

In [ ]:
addr

In [ ]:


pluto_path = 'C:\\Users\\comp\\Documents\\New York\\nyc_mappluto_20v3_fgdb\\MapPLUTO20v3.gdb'
#pluto = gpd.read_file("C:\\Users\\comp\\Documents\\New York\\nyc_mappluto_20v3_fgdb\\MapPLUTO20v3.gdb", driver='FileGDB')

layers = fiona.listlayers(pluto_path)

pluto = gpd.read_file(pluto_path, driver='FileGDB', layer = layers[0])

# for l in layers:
#     mallard = gpd.read_file('./bird-species/E00039600_mallard.gdb', layer=l)
#     mallard = mallard.to_crs(world.crs)
#     mallard.plot(ax=base, color='red', alpha=.1)

# # NATURAL GAS BY ZIPCODE

In [ ]:
ngc

In [ ]:
zip_shape = 'C:\\Users\\comp\\Documents\\New York\\ZIP_CODE_040114\\ZIP_CODE_040114.shp'
#df_zip = gpd.read_file(zip_shape)
#df_zip = df_zip.to_crs("EPSG:4326")
#df_zip.to_file('C:\\Users\\comp\\Documents\\New York\\zipcodes.geojson', driver='GeoJSON')

In [ ]:
df_zip_f = fiona.open(zip_shape,'r')
df_zip_f

In [ ]:
df_zip_f.crs

In [ ]:


df_zip_f = fiona.open(zip_shape,'r')
df_zip_f

import shapely

from pyproj import Proj, Transformer

transformer = Transformer.from_crs("epsg:2263", "epsg:4326")
transformer.transform(*coords[0][0])

df_zip_f.crs
zips = []
features = []
for rec in df_zip_f:
    rec['properties'] = dict(rec['properties'])
    # coordinates can have more than one shape
    for p in np.arange(len(rec['geometry']['coordinates'])):
        for c in np.arange(len(rec['geometry']['coordinates'][p])):
            x, y = transformer.transform(*rec['geometry']['coordinates'][p][c])
            rec['geometry']['coordinates'][p][c] = [y, x]
        features.append(rec)
        zips.append(rec['properties']['ZIPCODE'])

df_zip_f.close()


In [ ]:
np.arange(len(rec['geometry']['coordinates']))

In [ ]:
#features

In [ ]:
gj = {'type' : 'FeatureCollection', 'features' : features}
with open('test_output.geojson', 'w') as outfile:
    json.dump(gj, outfile)

In [ ]:
df_zips = pd.DataFrame(zips, columns = ['ZIPCODE'])
df_zips

In [ ]:
gj

In [ ]:
gj['features'][177]

In [ ]:
token = open("C:\\Users\\comp\\Documents\\New York\\token").read()

# GEOPANDAS IS STUPID AND CAUSES DEPENDENCY ISSUES

fig = px.choropleth_mapbox(df_zips, 
                    geojson=gj,
                    locations='ZIPCODE',
                    featureidkey= 'properties.ZIPCODE',
                    zoom=8, center = {"lat": 40, "lon": -73},)

# doesn't work with mapbox
#fig.update_geos(fitbounds="locations")
fig.update_layout(mapbox_style="light", mapbox_accesstoken = token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
np.arange(len(coords[0]))

In [ ]:
rec['geometry']['coordinates'][0]

In [ ]:
# from functools import partial
# import pyproj
# from shapely.ops import transform

# project = partial(
#     pyproj.transform,
#     pyproj.Proj(init='epsg:1727'), # source coordinate system
#     pyproj.Proj(init='epsg:4326')) # destination coordinate system

from pyproj import Proj, Transformer
p1 = Proj('epsg:2263')
p2 = Proj('epsg:4326')

transformer = Transformer.from_crs("epsg:2263", "epsg:4326")
transformer.transform(*coords[0][0])

#transform(p1, p2, coords[0])



In [ ]:
coords[0]


In [ ]:
df_zip.crs

In [ ]:
#import pyproj
#pyproj.Proj("+init=epsg:4326")


s = df_zip.loc[0,'geometry']
s
s.bounds


In [ ]:
from functools import partial
import pyproj
from shapely.ops import transform



project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:1727'), # source coordinate system
    pyproj.Proj(init='epsg:4326')) # destination coordinate system

#s2 = transform(project, s)  # apply projection

df_zip['geometry'] = df_zip['geometry'].apply(lambda x : transform(project, x))

In [ ]:
pyproj.Proj('PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",984250.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-74.0],PARAMETER["Standard_Parallel_1",40.66666666666666],PARAMETER["Standard_Parallel_2",41.03333333333333],PARAMETER["Latitude_Of_Origin",40.16666666666666],UNIT["Foot_US",0.3048006096012192]]')

In [ ]:
df_zip.head()

In [ ]:

#from pyproj import CRS
import pyproj
pyproj.crs.CRS.from_string("epsg:4326")

In [ ]:
df_zip.crs = "EPSG:4326"
df_zip.crs

In [ ]:
df_zip.plot()

In [ ]:
#df_zip.loc[0:1,'geometry'].to_crs("EPSG:4326")
df_zip.to_file('C:\\Users\\comp\\Documents\\New York\\zipcodes.geojson', driver='GeoJSON')

In [ ]:
fin = open('C:\\Users\\comp\\Documents\\New York\\zipcodes.geojson')
zip_json = json.load(fin)
fin.close()

In [ ]:
zip_json

In [ ]:
df_geoids = pd.DataFrame(zip_json['features'])
df_geoids2 = pd.json_normalize(df_geoids['properties'])
#df_geoids2
df_geoids2

In [ ]:
token = open("C:/Users/comp/Documents/intersection_congestion/default_token").read()

fig = px.choropleth(df_geoids2, geojson=zip_json,
                    locations='ZIPCODE',
                    featureidkey= 'properties.ZIPCODE')

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()